In [1]:
import sys
sys.path.append('../classes')

In [3]:
from DataLoader import DataLoaderFactory
import os

data_dir = "../../data/quickdraw_subset_np"

input_file_name = "train_images.npy"
label_file_name = "train_labels.npy"

inputs_path = os.path.join(data_dir, input_file_name)
labels_path = os.path.join(data_dir, label_file_name)

train_loader, val_loader = DataLoaderFactory.build(
    inputs_path, labels_path, val_ratio=0.1
)

In [ ]:
X, _ = train_loader.get_batch(1)

X = X / 255.0
print(X.shape)

(2, 28, 28)
